# Ettoday 網路爬蟲實作練習


* 能夠利用 Request + BeatifulSour 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例：

* 取出今天所有的新聞
* 取出今天下午三點到五點的新聞
* 根據範例，取出三天前下午三點到五點的新聞

In [67]:
import pandas as pd
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import re
from datetime import datetime

In [63]:
#extension
options = webdriver.ChromeOptions()
options.add_extension('/Users/mengchen/ChroPath.crx')

#open
url = 'https://www.ettoday.net/news/news-list.htm'
browser = webdriver.Chrome(executable_path='/Users/mengchen/chromedriver',options = options)
browser.get(url)
browser.maximize_window()

In [51]:
SCROLL_PAUSE_TIME = 0.5

# Get scroll height
last_height = browser.execute_script("return document.body.scrollHeight")

#HAPPY SCROLLING～～
while True:
    # Scroll down to bottom
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

## 根據範例，取出今天所有的新聞

In [52]:
result = browser.find_element_by_class_name('part_list_2')
news_res = result.get_attribute('innerHTML')
soup = BeautifulSoup(news_res)

all_news = soup.find_all('h3')

In [61]:
df = []
for news in all_news :
    if news.find('span',text=re.compile('2019/12/24.+')):
        df.append([news.find('span').text.split(' ')[0],news.find('span').text.split(' ')[1],news.find('em').text,news.find('a').text.replace('\u3000',' ')])
        
df = pd.DataFrame(df,columns=['date','time','category','title'])
df

,date,time,category,title
0,2019/12/24,16:07,財經,晉陞於部落設火箭場惹議...國研院幫背書：已向交通部申請一切合法
1,2019/12/24,16:05,旅遊,花蓮2020元旦舉辦「戰機衝場迎曙光」 看F-16戰機飛過太平洋
2,2019/12/24,16:03,國際,加強轟炸敘利亞「4天至少33死」！白頭盔：數十萬人將被迫逃離家園
3,2019/12/24,16:03,國際,英國「新年榮譽」獎表揚貢獻 體育界獲獎率比醫界高22倍
4,2019/12/24,16:00,新奇,「黑影男盪鞦韆」面屋還背屋？ 網友正反吵翻：看膝蓋彎曲
...,...,...,...,...
447,2019/12/24,00:12,旅遊,美樂蒂與雙子星變成下午茶甜點 台北文華東方度首度與三麗鷗合作
448,2019/12/24,00:10,社會,娃娃機台主遭8男亂棒打死！他頭骨碎裂倒血泊 18歲主嫌犯案流程曝光
449,2019/12/24,00:05,政治,蔡英文親女嬰照遭臉書審查 他怒罵「荒唐亂象只在民進黨執政發生」
450,2019/12/24,00:02,政治,韓國瑜連抱女嬰都被扭曲...議員齊轟：黑韓連BABY都不放過？


## 取出今天下午三點到五點的新聞

In [92]:
df[df.time.str.contains('(15|16):.+|17:00')]

,date,time,category,title
192,2019/12/21,17:00,旅遊,沒情人也暖！特搜高雄11家人氣麻油雞...第3家一開店就排隊
193,2019/12/21,16:59,政治,影／Wecare大遊行尹立鞠躬30秒 50萬人震撼宣言：現在是罷韓的起點
194,2019/12/21,16:59,政治,65萬人挺韓罷韓！高捷塞爆車廂月台都是人...總運量↑28% 高鐵旅客像過年
195,2019/12/21,16:59,國際,筆電插漏電插座！英女觸電「左手像在滾燙油裡」 痛一年染敗血症截肢
196,2019/12/21,16:59,國際,活貓塞微波爐加熱！ 2屁孩笑看牠「絕望抓門」錄影PO網引公憤
...,...,...,...,...
284,2019/12/21,15:02,影劇,小禎尾牙禮服「上下全露」！ 開U領「掉出半球深溝」嗨喊：賣胸季又到了
285,2019/12/21,15:00,生活,遠航持續停飛⋯原訂12/27接回ATR新機計劃「臨時喊卡」
286,2019/12/21,15:00,地方,斗六公園雲林耶誕城 民眾歡度閃亮聖誕夜
287,2019/12/21,15:00,社會,影／1221高捷爆滿！長裙妹疑被摸轉身狂巴 罷韓男大生無辜喊：是推擠！誰想摸你


## 根據範例，取出三天前下午三點到五點的新聞

In [80]:
today = datetime.today()
year = str(today.year)
month = str(today.month)
day = str(today.day-3)  #3天前

#year
selectYear = Select(browser.find_element_by_id("selY"))
selectYear.select_by_value(year)
#month
selectMonth = Select(browser.find_element_by_id("selM"))
selectMonth.select_by_value(month)
#day
selectDay = Select(browser.find_element_by_id("selD"))
selectDay.select_by_value(day)

#search btn
search_btn = browser.find_element_by_id('button')
search_btn.click()

In [81]:
SCROLL_PAUSE_TIME = 0.5

# Get scroll height
last_height = browser.execute_script("return document.body.scrollHeight")

#HAPPY SCROLLING～～
while True:
    # Scroll down to bottom
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [91]:
result = browser.find_element_by_class_name('part_list_2')
news_res = result.get_attribute('innerHTML')
soup = BeautifulSoup(news_res)
all_news = soup.find_all('h3')

df = []
for news in all_news :
    if news.find('span',text=re.compile(f'2019/12/{day}.+')):
        df.append([news.find('span').text.split(' ')[0],news.find('span').text.split(' ')[1],news.find('em').text,news.find('a').text.replace('\u3000',' ')])
        
df = pd.DataFrame(df,columns=['date','time','category','title'])

df[df.time.str.contains('(15|16):.+|17:00')]

,date,time,category,title
192,2019/12/21,17:00,旅遊,沒情人也暖！特搜高雄11家人氣麻油雞...第3家一開店就排隊
193,2019/12/21,16:59,政治,影／Wecare大遊行尹立鞠躬30秒 50萬人震撼宣言：現在是罷韓的起點
194,2019/12/21,16:59,政治,65萬人挺韓罷韓！高捷塞爆車廂月台都是人...總運量↑28% 高鐵旅客像過年
195,2019/12/21,16:59,國際,筆電插漏電插座！英女觸電「左手像在滾燙油裡」 痛一年染敗血症截肢
196,2019/12/21,16:59,國際,活貓塞微波爐加熱！ 2屁孩笑看牠「絕望抓門」錄影PO網引公憤
...,...,...,...,...
284,2019/12/21,15:02,影劇,小禎尾牙禮服「上下全露」！ 開U領「掉出半球深溝」嗨喊：賣胸季又到了
285,2019/12/21,15:00,生活,遠航持續停飛⋯原訂12/27接回ATR新機計劃「臨時喊卡」
286,2019/12/21,15:00,地方,斗六公園雲林耶誕城 民眾歡度閃亮聖誕夜
287,2019/12/21,15:00,社會,影／1221高捷爆滿！長裙妹疑被摸轉身狂巴 罷韓男大生無辜喊：是推擠！誰想摸你
